In [36]:
import pickle
from collections import defaultdict
import re
from dateutil.parser import parse
import spacy
import pandas as pd

In [3]:
with open("../reviewratings_iclr17_peeread.pkl", "rb") as f:
    rr = pickle.load(f)

In [4]:
# INSPECT_LIST SAMPLE:
# 
# {'abaselinefordetectingmisclassifiedandoutofdistributionexamplesinneuralnetworks': {'AnonReviewer1': {'MEANINGFUL_COMPARISON': [4,
#     datetime.datetime(2016, 12, 19, 0, 0)],
#    'ORIGINALITY': [4, datetime.datetime(2016, 12, 19, 0, 0)],
#    'RECOMMENDATION': [6, datetime.datetime(2016, 12, 19, 0, 0)]},
#   'AnonReviewer2': {'RECOMMENDATION': [6,
#     datetime.datetime(2016, 12, 16, 0, 0)]},
#   'AnonReviewer3': {'IMPACT': [2, datetime.datetime(2017, 1, 20, 0, 0)],
#    'ORIGINALITY': [3, datetime.datetime(2017, 1, 20, 0, 0)],
#    'RECOMMENDATION': [6, datetime.datetime(2017, 1, 20, 0, 0)],
#    'SOUNDNESS_CORRECTNESS': [3, datetime.datetime(2017, 1, 20, 0, 0)]}}

In [30]:
acl_aspects = ["MEANINGFUL_COMPARISON", "IMPACT", "ORIGINALITY", "RECOMMENDATION", "SUBSTANCE", "SOUNDNESS_CORRECTNESS", "APPROPRIATENESS", "CLARITY"]

aspect_dict = defaultdict(dict)
inspect_list = {}

for k in rr:
    new_key = rr[k]["title"].strip().lower()
    
    new_key = re.sub('[^0-9a-zA-Z]+', '', new_key)
    
    aspect_dict[new_key]["title"] = rr[k]["title"]
    aspect_dict[new_key]["dec"] = rr[k]["accepted"]
    aspect_dict[new_key]["pr_id"] = k
    
    inspect_list[new_key] = {}
    
    if "reviews" in rr[k]:
        for rev in rr[k]["reviews"]:
            if "OTHER_KEYS" in rev:
                if rev["OTHER_KEYS"].find("AnonReviewer") > -1:
                    anon_reviewer = rev["OTHER_KEYS"].split(" ")[-1]
                    if anon_reviewer in inspect_list[new_key]:
                        #inspect_list[new_key][anon_reviewer]
                        dummy = 1
                    else:
                        inspect_list[new_key][anon_reviewer] = {}
                    if "DATE" in rev:
                        if rev["DATE"].find("modified") > -1:
                            d = rev["DATE"]
                            try:
                                rev["DATE"] = re.findall('([0-9]+ [A-Za-z]+ [0-9][0-9][0-9][0-9]\))$', d)[0][:-1]
                            except Exception:
                                print(rev["DATE"])
                        else:
                            try:
                                rev["DATE"] = rev["DATE"].replace("(", "")
                                rev["DATE"] = rev["DATE"].replace(")", "")
                                rev["DATE"] = rev["DATE"].strip()
                                date_of_review = parse(rev["DATE"])
                            except Exception:
                                print(rev["DATE"], k)
                    else:
                        date_of_review = parse("1 Jan 2000")
                    
                    for asp in acl_aspects:
                        if asp in rev:
                            if asp in inspect_list[new_key][anon_reviewer]:
                                prev_date = inspect_list[new_key][anon_reviewer][asp][1]
                                if prev_date < date_of_review:
                                    prev_rev_text = inspect_list[new_key][anon_reviewer][asp][2]
                                    inspect_list[new_key][anon_reviewer][asp] = [rev[asp], date_of_review, rev["TITLE"] + ". " + rev["comments"] + prev_rev_text]
                                else:
                                    inspect_list[new_key][anon_reviewer][asp][2] = inspect_list[new_key][anon_reviewer][asp][2] + ". " +  rev["TITLE"] + ". " + rev["comments"]
                            else:
                                inspect_list[new_key][anon_reviewer][asp] = [rev[asp], date_of_review, rev["TITLE"] + ". " + rev["comments"], " "]

#                         if asp in rev:
#                             if asp in inspect_list[new_key][anon_reviewer]:
#                                 prev_date = inspect_list[new_key][anon_reviewer][asp][1]
#                                 if prev_date < date_of_review:
#                                     prev_rev_text = inspect_list[new_key][anon_reviewer][asp][2] + ". " + inspect_list[new_key][anon_reviewer][asp][3]
#                                     # current rev text, prev_rev_text
#                                     inspect_list[new_key][anon_reviewer][asp] = [rev[asp], date_of_review, rev["TITLE"] + ". " + rev["comments"], prev_rev_text]
#                                 else:
#                                     inspect_list[new_key][anon_reviewer][asp][3] = inspect_list[new_key][anon_reviewer][asp][3] + ". " +  rev["TITLE"] + ". " + rev["comments"]
#                             else:
#                                 inspect_list[new_key][anon_reviewer][asp] = [rev[asp], date_of_review, rev["TITLE"] + ". " + rev["comments"], " "]
            else:
                for asp in acl_aspects:
                    if asp in rev:
                        inspect_list.append((k, rev))

In [20]:
data_dir = "../../ICLR data/masterdata_unbalanced/"
year = 2017
paper_file = "papers_{}.pkl".format(year)


with open(data_dir+paper_file, "rb") as f:
    papers_data = pickle.load(f)


final_dict = {}

not_found = []

for k in papers_data:
    t = papers_data[k]["content"]["title"].strip().lower()
    t = re.sub('[^0-9a-zA-Z]+', '', t)
    if t in aspect_dict:
        final_dict[k] = aspect_dict[t]
        final_dict[k]["rev_info"] = inspect_list[t]
    else:
#         print(t, k)
        not_found.append(t)
print(len(not_found))

63


In [34]:
final_dict["H178hw9ex"]

{'dec': False,
 'pr_id': '618',
 'rev_info': {'AnonReviewer1': {'RECOMMENDATION': [5,
    datetime.datetime(2016, 12, 26, 0, 0),
    'review. This paper presents an improved formulation of CNN, aiming to separate geometric transformation from inherent features. The network can estimate the transformation of filters given the input images. \n\nThis work is based on a solid technical foundation and is motivated by a plausible rationale. Yet, the value of this work in practice is subject to questions:\n\n(1) It relies on the assumption that the input image is subject to a transformation on a certain Lie group (locally). Do such transformations constitute real challenges in practice? State-of-the-art CNNs, e.g. ResNet, are already quite resilient to such local deformations. What such components would add to the state of the art? Limited experiments on Cifar-10 does not seem to provide a very strong argument.\n\n(2) The computational cost is not discussed.',
    ' . review. This paper prese

In [21]:
nlp = spacy.load("en_core_web_sm")

In [61]:
excel_data = []
keywords = ["original", "proposal", "originality", "unique", "novelty", "novel", "contribution", "significance", "significant", "new way"]

for k in final_dict:
    if "rev_info" in final_dict[k]:
        for reviewer_num in final_dict[k]['rev_info']:
            if "ORIGINALITY" in final_dict[k]['rev_info'][reviewer_num]:
                #excel_data[k+"_"+reviewer_num] = { "dec": final_dict[k]["dec"],"score": final_dict[k]['rev_info'][reviewer_num]["ORIGINALITY"][0] }
                data_tuple = [k+"_"+reviewer_num, final_dict[k]["dec"], final_dict[k]['rev_info'][reviewer_num]["ORIGINALITY"][0]]

                current_text = final_dict[k]['rev_info'][reviewer_num]["ORIGINALITY"][2]
#                 other_text = final_dict[k]['rev_info'][reviewer_num]["ORIGINALITY"][3]

                doc = nlp(current_text)
                curr_comm = []
                for s in doc.sents:
                    sent_text = s.text.lower()
                    sent_text = sent_text.replace("\n", " ")
                    if sent_text.find("proposal") > -1 or (sent_text.find("proposed") > -1 and sent_text.find("new") > -1):
                        curr_comm.append(sent_text)
                    for kw in keywords:
                        if sent_text.find(kw) > -1:
                            curr_comm.append(sent_text)

#                 doc = nlp(other_text)
#                 other_comm = []
#                 for s in doc.sents:
#                     sent_text = s.text.lower()
#                     if sent_text.find("proposal") > -1 or (sent_text.find("proposed") > -1 and sent_text.find("new") > -1):
#                         other_comm.append(sent_text)
#                     for kw in keywords:
#                         if sent_text.find(kw) > -1:
#                             other_comm.append(sent_text)

                data_tuple.append(" ".join(curr_comm))
                data_tuple.append("https://openreview.net/forum?id={}".format(k))
                excel_data.append(data_tuple)
    else:
        print(k)

In [62]:
len(excel_data)

141

In [63]:
df = pd.DataFrame(excel_data, columns = ['Paper_ReviewId', 'Accept', 'NoveltyScore', "Comment", "OpenReview"])
print(df.head(2))
df.to_excel('2017_novelty_comments.xlsx', sheet_name='sheet1', index=False)

            Paper_ReviewId  Accept  NoveltyScore  \
0  SJJKxrsgl_AnonReviewer2    True             3   
1  BJrFC6ceg_AnonReviewer1    True             5   

                                             Comment  \
0                                                      
1  my summary of the main contribution: autoregre...   

                                  OpenReview  
0  https://openreview.net/forum?id=SJJKxrsgl  
1  https://openreview.net/forum?id=BJrFC6ceg  


## Scratch

In [2]:
with open("../features/rev_aspects_2017_peerread.pkl", "rb") as f:
    review_scores = pickle.load(f)

In [3]:
review_scores

{'B1-Hhnslg': {'RECOMMENDATION': [5, 4, 6],
  'dec': False,
  'pr_id': '559',
  'title': 'Prototypical Networks for Few-shot Learning'},
 'B1-q5Pqxl': {'CLARITY': [5, 5],
  'IMPACT': [3],
  'ORIGINALITY': [5],
  'RECOMMENDATION': [7, 6, 6],
  'SUBSTANCE': [5],
  'dec': True,
  'pr_id': '384',
  'title': 'Machine Comprehension Using Match-LSTM and Answer Pointer'},
 'B16Jem9xe': {'RECOMMENDATION': [8, 7],
  'dec': False,
  'pr_id': '534',
  'title': 'Learning in Implicit Generative Models'},
 'B16dGcqlx': {'CLARITY': [5],
  'ORIGINALITY': [1],
  'RECOMMENDATION': [6, 5],
  'SOUNDNESS_CORRECTNESS': [4],
  'dec': True,
  'pr_id': '341',
  'title': 'Third Person Imitation Learning'},
 'B184E5qee': {'CLARITY': [5],
  'IMPACT': [2],
  'ORIGINALITY': [2],
  'RECOMMENDATION': [5, 7, 9],
  'SOUNDNESS_CORRECTNESS': [4],
  'dec': True,
  'pr_id': '339',
  'title': 'Improving Neural Language Models with a Continuous Cache'},
 'B186cP9gx': {'RECOMMENDATION': [3, 4, 4],
  'dec': False,
  'pr_id': '6

In [4]:
review_scores["B16dGcqlx"]

{'CLARITY': [5],
 'ORIGINALITY': [1],
 'RECOMMENDATION': [6, 5],
 'SOUNDNESS_CORRECTNESS': [4],
 'dec': True,
 'pr_id': '341',
 'title': 'Third Person Imitation Learning'}

In [51]:
with open("../features/rev_aspects_2017_peerread.pkl", "rb") as f:
    review_scores = pickle.load(f)


In [52]:
for k in review_scores:
#     print(k, review_scores[k])
#     break
    if k == "SJJKxrsgl":
        print(review_scores[k])

{'title': 'Emergence of foveal image sampling from learning to attend in visual scenes', 'ORIGINALITY': [3], 'dec': True, 'RECOMMENDATION': [6, 5, 6], 'SOUNDNESS_CORRECTNESS': [5, 5], 'pr_id': '334', 'CLARITY': [5]}


In [53]:
with open("../reviewratings_iclr17_peeread.pkl", "rb") as f:
    rr = pickle.load(f)

In [54]:
rr['334']["reviews"]

[{'IS_META_REVIEW': True,
  'comments': "This paper proposed a neural attention model which has a learnable and differentiable sampling lattice. The work is well motivated as few previous work focus on learning the sampling lattice but with a fixed lattice. This work is quite similar to Spatial Transformer Networks (Jaderberg 2015), but the sampling lattice is learned by the model. The experiments showed that the model can learn a meaning lattice to the visual search task where the sampling lattice looks similar to human being's. \n\nThe main concern of the paper is that experiments are not sufficient. The paper only reports the results on a modified clustered MNIST dataset. It would be more interesting if the authors could conduct  the model on real datasets, such as Toronto Face dataset, CUB bird dataset and SVHN. For example, for the Face dataset, it would be nice if the model can learn to attend different parts of the face for expression recognition, or attend different part of bir